In [1]:
import pandas as pd

jan_data = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet')
feb_data = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet')

In [2]:
jan_data.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.3,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.9,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.9,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.1,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.4,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00


In [3]:
jan_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3066766 entries, 0 to 3066765
Data columns (total 19 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   VendorID               int64         
 1   tpep_pickup_datetime   datetime64[us]
 2   tpep_dropoff_datetime  datetime64[us]
 3   passenger_count        float64       
 4   trip_distance          float64       
 5   RatecodeID             float64       
 6   store_and_fwd_flag     object        
 7   PULocationID           int64         
 8   DOLocationID           int64         
 9   payment_type           int64         
 10  fare_amount            float64       
 11  extra                  float64       
 12  mta_tax                float64       
 13  tip_amount             float64       
 14  tolls_amount           float64       
 15  improvement_surcharge  float64       
 16  total_amount           float64       
 17  congestion_surcharge   float64       
 18  airport_fee           

## Number of columns for january

In [4]:
print(f"The number of columns for January is {len(jan_data.columns.to_list())}.")

The number of columns for January is 19.


## Calculating Duration

In [5]:
jan_data['duration'] = pd.to_datetime(jan_data.tpep_dropoff_datetime) - pd.to_datetime(jan_data.tpep_pickup_datetime)
jan_data['duration'] = jan_data['duration'].dt.total_seconds() / 60

In [6]:
print(f"The duration for January is {jan_data['duration'].mean()}.")

The duration for January is 15.668995167330452.


## removing outlier

In [7]:
clean_jan_data = jan_data[(jan_data.duration >= 1) & (jan_data.duration <= 60)]

In [8]:
print(f"The percentage of data left after removing outlier is {len(clean_jan_data)/len(jan_data)*100:.2f}%.")

The percentage of data left after removing outlier is 98.12%.


## one hot encoding

In [9]:
# # apply one-hot encoding to the pickup and dropoff location IDs.
# Turn the dataframe into a list of dictionaries (remember to re-cast the ids to strings - otherwise it will label encode them)
# Fit a dictionary vectorizer
# Get a feature matrix from it

In [10]:
from sklearn.feature_extraction import DictVectorizer

categorical = ['PULocationID', 'DOLocationID']


clean_jan_data['PULocationID'] = clean_jan_data['PULocationID'].astype(str)
clean_jan_data['DOLocationID'] = clean_jan_data['DOLocationID'].astype(str)

records = clean_jan_data[['PULocationID', 'DOLocationID']].to_dict(orient='records')

vec = DictVectorizer(sparse=True)  #sparse=True for large datasets
feature_matrix = vec.fit_transform(records)

<ipython-input-10-d0ef51e60c25>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_jan_data['PULocationID'] = clean_jan_data['PULocationID'].astype(str)
<ipython-input-10-d0ef51e60c25>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_jan_data['DOLocationID'] = clean_jan_data['DOLocationID'].astype(str)


In [12]:
print(f"The shape of feature matrix after one hot encoding is {feature_matrix.shape[1]}")

The shape of feature matrix after one hot encoding is 515


## Training a plain Linear Regression model

In [16]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import numpy as np

X_train, X_val, y_train, y_val = train_test_split(
    feature_matrix,
    clean_jan_data['duration'].values,
    test_size=0.2,
    random_state=42
)

lr = LinearRegression()
lr.fit(X_train, y_train)

y_train_pred = lr.predict(X_train)
y_val_pred = lr.predict(X_val)

In [17]:
def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

print(f"Train RMSE: {rmse(y_train, y_train_pred):.2f}")
print(f"Validation RMSE: {rmse(y_val, y_val_pred):.2f}")

Train RMSE: 7.65
Validation RMSE: 7.66
